
<br> <br> <br> <br>
<br> <br> <br> <br>

<center>


### EAI 6000 Fundamentals of AI
### Assignment 3 Tree Based Models
### Shivangi Vashi
### Anirudh Sharma
### Sandeep Srivastava
### Sweta Mankala


<br> <br> <br> <br>
<br> <br> <br> <br>
  

### Northeastern University
### Instructor: Kasun Samarasinghe
### Fall 2020
### 12 November 2020
</center>

******
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>

<style>
body {
text-align: justify
font-size: 16pt}
</style>


## <center>Introduction  </center>

<br><br>
This week we explore Tree based models. Our data is from: [policing data](https://openpolicing.stanford.edu/data/), and we look at Massachusetts Traffic Stops. Our goal is to explore policing and understand factors that affect the decisions made by the police. In this analysis, we will be performing 
<br><br>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from IPython.display import display

In [2]:
#Importing data
openpolicing_path="../ma_statewide_2020_04_01.csv"

data=pd.read_csv(openpolicing_path,low_memory=False)

In [3]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


We check the data types to see if appropriate types are being used. We will convert them accordinly.

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', data.shape[0]+1)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   raw_row_number              int64  
 1   date                        object 
 2   location                    object 
 3   county_name                 object 
 4   subject_age                 float64
 5   subject_race                object 
 6   subject_sex                 object 
 7   type                        object 
 8   arrest_made                 object 
 9   citation_issued             object 
 10  warning_issued              object 
 11  outcome                     object 
 12  contraband_found            object 
 13  contraband_drugs            object 
 14  contraband_weapons          object 
 15  contraband_alcohol          bool   
 16  contraband_other            object 
 17  frisk_performed             object 
 18  search_conducted            bool   
 19  search_basis         

We convert the date from object to date type and extract time for future analyses.

In [8]:
##Convert date to datetype
data.date
data["date"] = pd.to_datetime(data.date, format="%Y-%m-%d")

In [9]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,False,citation,NaN,NaN,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,True,warning,False,False,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,True,warning,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,True,warning,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,False,citation,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


In [10]:
#Convering race, sex, location etc to categorical
data.iloc[:,np.r_[2:4,5:8,9:12,19:23]]=data.iloc[:,np.r_[2:4,5:8,9:12,19:23]].astype("category")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   raw_row_number              int64         
 1   date                        datetime64[ns]
 2   location                    category      
 3   county_name                 category      
 4   subject_age                 float64       
 5   subject_race                category      
 6   subject_sex                 category      
 7   type                        category      
 8   arrest_made                 object        
 9   citation_issued             category      
 10  warning_issued              category      
 11  outcome                     category      
 12  contraband_found            object        
 13  contraband_drugs            object        
 14  contraband_weapons          object        
 15  contraband_alcohol          bool          
 16  contraband_other  

In [11]:
#Checking proportion of missing values
data.isnull().sum()*100/len(data)

raw_row_number                 0.000000
date                           0.000000
location                       0.195127
county_name                    0.195127
subject_age                    4.625146
subject_race                   0.048709
subject_sex                    0.457316
type                           0.000000
arrest_made                    0.026813
citation_issued                0.026813
warning_issued                 0.026813
outcome                        0.200747
contraband_found              98.368673
contraband_drugs              98.368673
contraband_weapons            98.368673
contraband_alcohol             0.000000
contraband_other              98.368673
frisk_performed               98.400843
search_conducted               0.000000
search_basis                  98.506281
reason_for_stop               48.579432
vehicle_type                   0.145277
vehicle_registration_state     0.287275
raw_Race                       0.048709
dtype: float64

<!-- We will use the KNN Imputer for imputing values into the Age and Sex variables for better precision -->

We remove nulls from the race column since they constitute only 0.048% of the data for better accuracy.

In [12]:
#Removing null, and unknown values from race
data=data[-data["subject_race"].isin(["other","unknown",np.nan])]
data["subject_race"].unique()

[white, hispanic, black, asian/pacific islander]
Categories (4, object): [white, hispanic, black, asian/pacific islander]

In [14]:
data.nunique()

raw_row_number                3386549
date                             3287
location                          350
county_name                        14
subject_age                        85
subject_race                        4
subject_sex                         2
type                                1
arrest_made                         2
citation_issued                     2
warning_issued                      2
outcome                             3
contraband_found                    2
contraband_drugs                    2
contraband_weapons                  2
contraband_alcohol                  2
contraband_other                    2
frisk_performed                     2
search_conducted                    2
search_basis                        3
reason_for_stop                     7
vehicle_type                        5
vehicle_registration_state         51
raw_Race                            5
dtype: int64

In [15]:
# Removing row number 
data.drop(['raw_row_number'], axis=1, inplace = True)

Encoding categorical variables for future analyses

In [16]:
data.dtypes

date                          datetime64[ns]
location                            category
county_name                         category
subject_age                          float64
subject_race                        category
subject_sex                         category
type                                category
arrest_made                           object
citation_issued                     category
warning_issued                      category
outcome                             category
contraband_found                      object
contraband_drugs                      object
contraband_weapons                    object
contraband_alcohol                      bool
contraband_other                      object
frisk_performed                       object
search_conducted                        bool
search_basis                        category
reason_for_stop                     category
vehicle_type                        category
vehicle_registration_state          category
raw_Race  

In [18]:
data.select_dtypes(include=["object"]).nunique()

arrest_made           2
contraband_found      2
contraband_drugs      2
contraband_weapons    2
contraband_other      2
frisk_performed       2
raw_Race              5
dtype: int64

In [40]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

print(data['month'].head)
data.drop(['date'], axis=1, inplace = True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Encoding categorical variables

We use the get_dummies function to convert categorical variables to one hot encoded indicator variables

In [ ]:
#One hot encoding categorical variables

pd.get_dummies(data.loc[:,['subject_race','subject_sex','arrest_made','citation_issued','outcome','contraband_found','contraband_drugs','contraband_weapons','contraband_alcohol','contraband_other','search_basis','reason_for_stop']])

In [21]:
pd.get_dummies(data).head()

,date,subject_age,contraband_alcohol,search_conducted,location_ABINGTON,location_ACTON,location_ACUSHNET,location_ADAMS,location_AGAWAM,location_ALFORD,location_AMESBURY,location_AMHERST,location_ANDOVER,location_ARLINGTON,location_ASHBURNHAM,location_ASHBY,location_ASHFIELD,location_ASHLAND,location_ATHOL,location_ATTLEBORO,location_AUBURN,location_AVON,location_AYER,location_BARNSTABLE,location_BARRE,location_BECKET,location_BEDFORD,location_BELCHERTOWN,location_BELLINGHAM,location_BELMONT,location_BERKLEY,location_BERLIN,location_BERNARDSTON,location_BEVERLY,location_BILLERICA,location_BLACKSTONE,location_BLANDFORD,location_BOLTON,location_BOSTON,location_BOURNE,location_BOXBOROUGH,location_BOXFORD,location_BOYLSTON,location_BRAINTREE,location_BREWSTER,location_BRIDGEWATER,location_BRIMFIELD,location_BROCKTON,location_BROOKFIELD,location_BROOKLINE,location_BUCKLAND,location_BURLINGTON,location_CAMBRIDGE,location_CANTON,location_CARLISLE,location_CARVER,location_CHARLEMONT,location_CHARLTON,location_CHATHAM,location_CHELMSFORD,location_CHELSEA,location_CHESHIRE,location_CHESTER,location_CHESTERFIELD,location_CHICOPEE,location_CHILMARK,location_CLARKSBURG,location_CLINTON,location_COHASSET,location_COLRAIN,location_CONCORD,location_CONWAY,location_CUMMINGTON,location_DALTON,location_DANVERS,location_DARTMOUTH,location_DEDHAM,location_DEERFIELD,location_DENNIS,location_DIGHTON,location_DOUGLAS,location_DOVER,location_DRACUT,location_DUDLEY,location_DUNSTABLE,location_DUXBURY,location_EAST BRIDGEWATER,location_EAST BROOKFIELD,location_EAST LONGMEADOW,location_EASTHAM,location_EASTHAMPTON,location_EASTON,location_EDGARTOWN,location_EGREMONT,location_ERVING,location_ESSEX,location_EVERETT,location_FAIRHAVEN,location_FALL RIVER,location_FALMOUTH,location_FITCHBURG,location_FLORIDA,location_FOXBOROUGH,location_FRAMINGHAM,location_FRANKLIN,location_FREETOWN,location_GARDNER,location_GAY HEAD/AQUINNAH,location_GEORGETOWN,location_GILL,location_GLOUCESTER,location_GOSHEN,location_GRAFTON,location_GRANBY,location_GRANVILLE,location_GREAT BARRINGTON,location_GREENFIELD,location_GROTON,location_GROVELAND,location_HADLEY,location_HALIFAX,location_HAMILTON,location_HAMPDEN,location_HANCOCK,location_HANOVER,location_HANSON,location_HARDWICK,location_HARVARD,location_HARWICH,location_HATFIELD,location_HAVERHILL,location_HAWLEY,location_HEATH,location_HINGHAM,location_HINSDALE,location_HOLBROOK,location_HOLDEN,location_HOLLAND,location_HOLLISTON,location_HOLYOKE,location_HOPEDALE,location_HOPKINTON,location_HUBBARDSTON,location_HUDSON,location_HULL,location_HUNTINGTON,location_IPSWICH,location_KINGSTON,location_LAKEVILLE,location_LANCASTER,location_LANESBOROUGH,location_LAWRENCE,location_LEE,location_LEICESTER,location_LENOX,location_LEOMINSTER,location_LEVERETT,location_LEXINGTON,location_LEYDEN,location_LINCOLN,location_LITTLETON,location_LONGMEADOW,location_LOWELL,location_LUDLOW,location_LUNENBURG,location_LYNN,location_LYNNFIELD,location_MALDEN,location_MANCHESTER,location_MANSFIELD,location_MARBLEHEAD,location_MARION,location_MARLBOROUGH,location_MARSHFIELD,location_MASHPEE,location_MATTAPOISETT,location_MAYNARD,location_MEDFIELD,location_MEDFORD,location_MEDWAY,location_MELROSE,location_MENDON,location_MERRIMAC,location_METHUEN,location_MIDDLEBOROUGH,location_MIDDLEFIELD,location_MIDDLETON,location_MILFORD,location_MILLBURY,location_MILLIS,location_MILLVILLE,location_MILTON,location_MONROE,location_MONSON,location_MONTAGUE,location_MONTEREY,location_MONTGOMERY,location_MOUNT WASHINGTON,location_NAHANT,location_NANTUCKET,location_NATICK,location_NEEDHAM,location_NEW ASHFORD,location_NEW BEDFORD,location_NEW BRAINTREE,location_NEW MARLBOROUGH,location_NEW SALEM,location_NEWBURY,location_NEWBURYPORT,location_NEWTON,location_NORFOLK,location_NORTH ADAMS,location_NORTH ANDOVER,location_NORTH ATTLEBOROUGH,location_NORTH BROOKFIELD,location_NORTH READING,location_NORTHAMPTON,location_NORTHBOROUGH,location_NORTHBRIDGE,location_NORTHFIELD,locat

In [23]:
# Before One-Hot Encoding, let's create separate indep. X variables from dep. Y variable
X = data.drop('subject_race', axis = 1).copy()
# alternatively, X = df.iloc[:, :-1]
y = data['subject_race'].copy()

In [24]:
X_enc = pd.get_dummies(X)
display(X_enc.head())
display(X.shape)
display(X_enc.shape)

,date,subject_age,contraband_alcohol,search_conducted,location_ABINGTON,location_ACTON,location_ACUSHNET,location_ADAMS,location_AGAWAM,location_ALFORD,location_AMESBURY,location_AMHERST,location_ANDOVER,location_ARLINGTON,location_ASHBURNHAM,location_ASHBY,location_ASHFIELD,location_ASHLAND,location_ATHOL,location_ATTLEBORO,location_AUBURN,location_AVON,location_AYER,location_BARNSTABLE,location_BARRE,location_BECKET,location_BEDFORD,location_BELCHERTOWN,location_BELLINGHAM,location_BELMONT,location_BERKLEY,location_BERLIN,location_BERNARDSTON,location_BEVERLY,location_BILLERICA,location_BLACKSTONE,location_BLANDFORD,location_BOLTON,location_BOSTON,location_BOURNE,location_BOXBOROUGH,location_BOXFORD,location_BOYLSTON,location_BRAINTREE,location_BREWSTER,location_BRIDGEWATER,location_BRIMFIELD,location_BROCKTON,location_BROOKFIELD,location_BROOKLINE,location_BUCKLAND,location_BURLINGTON,location_CAMBRIDGE,location_CANTON,location_CARLISLE,location_CARVER,location_CHARLEMONT,location_CHARLTON,location_CHATHAM,location_CHELMSFORD,location_CHELSEA,location_CHESHIRE,location_CHESTER,location_CHESTERFIELD,location_CHICOPEE,location_CHILMARK,location_CLARKSBURG,location_CLINTON,location_COHASSET,location_COLRAIN,location_CONCORD,location_CONWAY,location_CUMMINGTON,location_DALTON,location_DANVERS,location_DARTMOUTH,location_DEDHAM,location_DEERFIELD,location_DENNIS,location_DIGHTON,location_DOUGLAS,location_DOVER,location_DRACUT,location_DUDLEY,location_DUNSTABLE,location_DUXBURY,location_EAST BRIDGEWATER,location_EAST BROOKFIELD,location_EAST LONGMEADOW,location_EASTHAM,location_EASTHAMPTON,location_EASTON,location_EDGARTOWN,location_EGREMONT,location_ERVING,location_ESSEX,location_EVERETT,location_FAIRHAVEN,location_FALL RIVER,location_FALMOUTH,location_FITCHBURG,location_FLORIDA,location_FOXBOROUGH,location_FRAMINGHAM,location_FRANKLIN,location_FREETOWN,location_GARDNER,location_GAY HEAD/AQUINNAH,location_GEORGETOWN,location_GILL,location_GLOUCESTER,location_GOSHEN,location_GRAFTON,location_GRANBY,location_GRANVILLE,location_GREAT BARRINGTON,location_GREENFIELD,location_GROTON,location_GROVELAND,location_HADLEY,location_HALIFAX,location_HAMILTON,location_HAMPDEN,location_HANCOCK,location_HANOVER,location_HANSON,location_HARDWICK,location_HARVARD,location_HARWICH,location_HATFIELD,location_HAVERHILL,location_HAWLEY,location_HEATH,location_HINGHAM,location_HINSDALE,location_HOLBROOK,location_HOLDEN,location_HOLLAND,location_HOLLISTON,location_HOLYOKE,location_HOPEDALE,location_HOPKINTON,location_HUBBARDSTON,location_HUDSON,location_HULL,location_HUNTINGTON,location_IPSWICH,location_KINGSTON,location_LAKEVILLE,location_LANCASTER,location_LANESBOROUGH,location_LAWRENCE,location_LEE,location_LEICESTER,location_LENOX,location_LEOMINSTER,location_LEVERETT,location_LEXINGTON,location_LEYDEN,location_LINCOLN,location_LITTLETON,location_LONGMEADOW,location_LOWELL,location_LUDLOW,location_LUNENBURG,location_LYNN,location_LYNNFIELD,location_MALDEN,location_MANCHESTER,location_MANSFIELD,location_MARBLEHEAD,location_MARION,location_MARLBOROUGH,location_MARSHFIELD,location_MASHPEE,location_MATTAPOISETT,location_MAYNARD,location_MEDFIELD,location_MEDFORD,location_MEDWAY,location_MELROSE,location_MENDON,location_MERRIMAC,location_METHUEN,location_MIDDLEBOROUGH,location_MIDDLEFIELD,location_MIDDLETON,location_MILFORD,location_MILLBURY,location_MILLIS,location_MILLVILLE,location_MILTON,location_MONROE,location_MONSON,location_MONTAGUE,location_MONTEREY,location_MONTGOMERY,location_MOUNT WASHINGTON,location_NAHANT,location_NANTUCKET,location_NATICK,location_NEEDHAM,location_NEW ASHFORD,location_NEW BEDFORD,location_NEW BRAINTREE,location_NEW MARLBOROUGH,location_NEW SALEM,location_NEWBURY,location_NEWBURYPORT,location_NEWTON,location_NORFOLK,location_NORTH ADAMS,location_NORTH ANDOVER,location_NORTH ATTLEBOROUGH,location_NORTH BROOKFIELD,location_NORTH READING,location_NORTHAMPTON,location_NORTHBOROUGH,location_NORTHBRIDGE,location_NORTHFIELD,locat

(3386549, 22)

(3386549, 463)

In [25]:
# Make sure to convert y dep. variable to from categorical to continous
display(y.value_counts())

white                     2529780
black                      351610
hispanic                   338317
asian/pacific islander     166842
unknown                         0
other                           0
Name: subject_race, dtype: int64

In [29]:
# encode string class values as integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

In [37]:
# Checking if the data is imbalanced or not
sum(label_encoded_y)/ len(label_encoded_y)

2.5446506163058618

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, label_encoded_y, random_state = 42, stratify = label_encoded_y)
print("train ratio =", sum(y_train)/ len(y_train))
print("test ratio =", sum(y_test)/ len(y_test))


train ratio = 2.5446501865616553
test ratio = 2.544651905536959


In [39]:
# Creating the XGBClassfier shell
clf_xgb = xgb.XGBClassifier(objective='multi:softprob', missing=None, seed=42)
# Adding the data and fitting the model
# Instead of finding the optimal # of tress using K-Cross Validation, let's use early stopping to stop the tree when the cost function no longer reduces
# XGBoost will do the cross validation for us; we just have to specifiy the # of rounds to exhaust with no improvement before stopping
# We use evaluation metric as Area Under Precision-Recall Curve
clf_xgb.fit(X_train,
            y_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='aucpr',
            eval_set=[(X_test, y_test)])

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields date

<!-- We create a new binomial variable that tells us if the stop was before or after sunset -->